In [1]:
%load_ext autoreload
%autoreload 2
import os
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')

# Read and Decode BBCI Data

This tutorial shows how to read and decode BBCI data.

## Setup logging to see outputs

In [2]:
import logging
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)
log = logging.getLogger()

## Load and preprocess data

First set the filename and the sensors you want to load. If you set

```python
load_sensor_names=None
```

or just remove the parameter from the function call, all sensors will be loaded.

In [3]:
from braindecode.datasets.bbci import BBCIDataset
train_filename = '/home/schirrmr/data/BBCI-without-last-runs/BhNoMoSc1S001R01_ds10_1-12.BBCI.mat'
cnt = BBCIDataset(train_filename, load_sensor_names=['C3', 'CPz', 'C4']).load()

Creating RawArray with float64 data, n_channels=3, n_times=3451320
    Range : 0 ... 3451319 =      0.000 ...  6902.638 secs
Ready.


### Preprocessing on continous data

First remove the stimulus channel, than apply any preprocessing you like. There are some very few directions available from Braindecode, such as resample_cnt. But you can apply any function on the chan x time matrix of the mne raw object (`cnt` in the code) by calling `mne_apply` with two arguments:

1. Your function (2d-array-> 2darray), that transforms the channel x timesteps data array
2. the Raw data object from mne itself

In [4]:
from braindecode.mne_ext.signalproc import resample_cnt, mne_apply
from braindecode.datautil.signalproc import exponential_running_standardize
# Remove stimulus channel
cnt = cnt.drop_channels(['STI 014'])
cnt = resample_cnt(cnt, 250)
# mne apply will apply the function to the data (a 2d-numpy-array)
# have to transpose data back and forth, since
# exponential_running_standardize expects time x chans order
# while mne object has chans x time order
cnt = mne_apply(lambda a: exponential_running_standardize(
    a.T, init_block_size=1000,factor_new=0.001, eps=1e-4).T,
    cnt)

2017-11-03 17:59:09,513 WARNING : This is not causal, uses future data....
2017-11-03 17:59:09,514 INFO : Resampling from 500.000000 to 250.000000 Hz.
Creating RawArray with float64 data, n_channels=3, n_times=1725660
    Range : 0 ... 1725659 =      0.000 ...  6902.636 secs
Ready.


## Transform to epoched dataset 

Braindecode supplies the `create_signal_target_from_raw_mne` function, which will transform the mne raw object into a `SignalAndTarget` object for use in Braindecode.
`name_to_code` should be an `OrderedDict` that maps class names to either one or a list of marker codes for that class.

In [5]:
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne
from collections import OrderedDict
# can also give lists of marker codes in case a class has multiple marker codes...
name_to_code = OrderedDict([('Right', 1), ('Left', 2), ('Rest', 3), ('Feet', 4)])
segment_ival_ms = [-500,4000]

train_set = create_signal_target_from_raw_mne(cnt, name_to_code, segment_ival_ms)

2017-11-03 17:59:11,376 INFO : Trial per class:
Counter({'Feet': 225, 'Right': 224, 'Rest': 224, 'Left': 224})


## Same for test set

In [6]:
test_filename = '/home/schirrmr/data/BBCI-only-last-runs/BhNoMoSc1S001R13_ds10_1-2BBCI.mat'
cnt = BBCIDataset(test_filename, load_sensor_names=['C3', 'CPz', 'C4']).load()
# Remove stimulus channel
cnt = cnt.drop_channels(['STI 014'])
cnt = resample_cnt(cnt, 250)
cnt = mne_apply(lambda a: exponential_running_standardize(
    a.T, init_block_size=1000,factor_new=0.001, eps=1e-4).T,
    cnt)
test_set = create_signal_target_from_raw_mne(cnt, name_to_code, segment_ival_ms)

Creating RawArray with float64 data, n_channels=3, n_times=617090
    Range : 0 ... 617089 =      0.000 ...  1234.178 secs
Ready.
2017-11-03 17:59:12,305 WARNING : This is not causal, uses future data....
2017-11-03 17:59:12,306 INFO : Resampling from 500.000000 to 250.000000 Hz.
Creating RawArray with float64 data, n_channels=3, n_times=308545
    Range : 0 ... 308544 =      0.000 ...  1234.176 secs
Ready.
2017-11-03 17:59:12,653 INFO : Trial per class:
Counter({'Feet': 40, 'Left': 40, 'Rest': 40, 'Right': 40})


<div class="alert alert-info">

In case of start and stop markers, provide a `name_to_stop_codes` dictionary (same as for the start codes in this example) as a final argument to `create_signal_target_from_raw_mne`. See [Read and Decode BBCI Data with Start-Stop-Markers Tutorial](BBCI_Data_Start_Stop.html)


</div>

Split off a validation set.

In [7]:
from braindecode.datautil.splitters import split_into_two_sets

train_set, valid_set = split_into_two_sets(train_set, first_set_fraction=0.8)


## Create the model

In [8]:
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = True
set_random_seeds(seed=20170629, cuda=cuda)


# This will determine how many crops are processed in parallel
input_time_length = train_set.X.shape[2]
in_chans = 3
n_classes = 4
# final_conv_length determines the size of the receptive field of the ConvNet
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes, input_time_length=input_time_length,
                        final_conv_length='auto').create_network()

if cuda:
    model.cuda()

## Setup optimizer and iterator

In [9]:
from torch import optim
import numpy as np

optimizer = optim.Adam(model.parameters())


from braindecode.datautil.iterators import BalancedBatchSizeIterator
iterator = BalancedBatchSizeIterator(batch_size=32)


## Setup Monitors, Loss function, Stop Criteria

In [10]:
from braindecode.experiments.experiment import Experiment
from braindecode.experiments.monitors import RuntimeMonitor, LossMonitor, CroppedTrialMisclassMonitor, MisclassMonitor
from braindecode.experiments.stopcriteria import MaxEpochs
import torch.nn.functional as F
import torch as th
from braindecode.torch_ext.modules import Expression


loss_function = F.nll_loss

model_constraint = None
monitors = [LossMonitor(), MisclassMonitor(col_suffix='misclass'), 
            RuntimeMonitor(),]
stop_criterion = MaxEpochs(20)
exp = Experiment(model, train_set, valid_set, test_set, iterator, loss_function, optimizer, model_constraint,
          monitors, stop_criterion, remember_best_column='valid_misclass',
          run_after_early_stop=True, batch_modifier=None, cuda=cuda)

## Run experiment

In [11]:
exp.run()

2017-11-03 17:59:15,468 INFO : Run until first stop...
2017-11-03 17:59:15,979 INFO : Epoch 0
2017-11-03 17:59:15,981 INFO : train_loss                3.82544
2017-11-03 17:59:15,982 INFO : valid_loss                3.72206
2017-11-03 17:59:15,982 INFO : test_loss                 3.90394
2017-11-03 17:59:15,983 INFO : train_misclass            0.76184
2017-11-03 17:59:15,984 INFO : valid_misclass            0.76536
2017-11-03 17:59:15,985 INFO : test_misclass             0.75625
2017-11-03 17:59:15,985 INFO : runtime                   0.00000
2017-11-03 17:59:15,986 INFO : 
2017-11-03 17:59:15,988 INFO : New best valid_misclass: 0.765363
2017-11-03 17:59:15,989 INFO : 
2017-11-03 17:59:16,564 INFO : Time only for training updates: 0.57s
2017-11-03 17:59:16,671 INFO : Epoch 1
2017-11-03 17:59:16,672 INFO : train_loss                2.65436
2017-11-03 17:59:16,673 INFO : valid_loss                2.88156
2017-11-03 17:59:16,674 INFO : test_loss                 3.03544
2017-11-03 17:59:16

2017-11-03 17:59:23,976 INFO : test_loss                 0.78449
2017-11-03 17:59:23,977 INFO : train_misclass            0.05710
2017-11-03 17:59:23,977 INFO : valid_misclass            0.18994
2017-11-03 17:59:23,978 INFO : test_misclass             0.30000
2017-11-03 17:59:23,979 INFO : runtime                   0.66576
2017-11-03 17:59:23,980 INFO : 
2017-11-03 17:59:24,527 INFO : Time only for training updates: 0.55s
2017-11-03 17:59:24,636 INFO : Epoch 13
2017-11-03 17:59:24,637 INFO : train_loss                0.22913
2017-11-03 17:59:24,638 INFO : valid_loss                0.52561
2017-11-03 17:59:24,639 INFO : test_loss                 0.68987
2017-11-03 17:59:24,639 INFO : train_misclass            0.06407
2017-11-03 17:59:24,640 INFO : valid_misclass            0.18436
2017-11-03 17:59:24,641 INFO : test_misclass             0.26250
2017-11-03 17:59:24,642 INFO : runtime                   0.66327
2017-11-03 17:59:24,642 INFO : 
2017-11-03 17:59:24,645 INFO : New best valid_m

2017-11-03 17:59:32,106 INFO : test_misclass             0.29375
2017-11-03 17:59:32,107 INFO : runtime                   0.67182
2017-11-03 17:59:32,108 INFO : 
2017-11-03 17:59:32,648 INFO : Time only for training updates: 0.54s
2017-11-03 17:59:32,772 INFO : Epoch 26
2017-11-03 17:59:32,773 INFO : train_loss                0.13316
2017-11-03 17:59:32,774 INFO : valid_loss                0.21726
2017-11-03 17:59:32,775 INFO : test_loss                 0.72940
2017-11-03 17:59:32,776 INFO : train_misclass            0.04571
2017-11-03 17:59:32,776 INFO : valid_misclass            0.08380
2017-11-03 17:59:32,777 INFO : test_misclass             0.27500
2017-11-03 17:59:32,778 INFO : runtime                   0.67151
2017-11-03 17:59:32,779 INFO : 


We arrive around 26%, exact value depending on stars :))